In [1]:
pip install imutils

In [2]:
# import the necessary packages
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils.video import VideoStream
import numpy as np
import imutils
import time
import cv2
import os

def detect_and_predict_mask(frame, faceNet, maskNet):
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1.0, (224, 224),(104.0, 177.0, 123.0))
    faceNet.setInput(blob)
    detections = faceNet.forward()
    print(detections.shape)
    faces = []
    locs = []
    preds = []
    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.5:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
            (startX, startY) = (max(0, startX), max(0, startY))
            (endX, endY) = (min(w - 1, endX), min(h - 1, endY))
            face = frame[startY:endY, startX:endX]
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face = cv2.resize(face, (224, 224))
            face = img_to_array(face)
            face = preprocess_input(face)
            faces.append(face)
            locs.append((startX, startY, endX, endY))
    if len(faces) > 0:
        faces = np.array(faces, dtype="float32")
        preds = maskNet.predict(faces, batch_size=32)
    return (locs, preds)

prototxtPath = r"face_detector\deploy.prototxt"
weightsPath = r"face_detector\res10_300x300_ssd_iter_140000.caffemodel"
faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)
maskNet = load_model("face_mask_detector.h5")
print("[INFO] starting video stream...")
vs = VideoStream(src=0).start()

# loop over the frames from the video stream
while True:
    frame = vs.read()
    frame = imutils.resize(frame, width=224)
    (locs, preds) = detect_and_predict_mask(frame, faceNet, maskNet)
    for (box, pred) in zip(locs, preds):
        (startX, startY, endX, endY) = box
        (mask, withoutMask) = pred
        print(mask,withoutMask)
        label = "Mask" if mask > withoutMask else "No Mask"
        color = (0, 255, 0) if label == "Mask" else (0, 0, 255)
        label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)
        cv2.putText(frame, label, (startX, startY - 10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
        cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break

# do a bit of cleanup
cv2.destroyAllWindows()
vs.stop()

[INFO] starting video stream...
(1, 1, 114, 7)
1/1 [==============================] - 1s 539ms/step
1.0 7.1103207e-16
(1, 1, 114, 7)
1/1 [==============================] - 0s 76ms/step
1.0 2.0623968e-16
(1, 1, 118, 7)
1/1 [==============================] - 0s 69ms/step
1.0 3.7450162e-17
(1, 1, 114, 7)
1/1 [==============================] - 0s 52ms/step
1.0 4.147678e-18
(1, 1, 117, 7)
1/1 [==============================] - 0s 65ms/step
1.0 2.7591786e-18
(1, 1, 110, 7)
1/1 [==============================] - 0s 73ms/step
1.0 4.4830203e-16
(1, 1, 121, 7)
1/1 [==============================] - 0s 64ms/step
1.0 6.2452724e-16
(1, 1, 112, 7)
1/1 [==============================] - 0s 64ms/step
1.0 4.6497757e-16
(1, 1, 112, 7)
1/1 [==============================] - 0s 63ms/step
1.0 1.3821791e-16
(1, 1, 109, 7)
1/1 [==============================] - 0s 54ms/step
1.0 1.7782287e-17
(1, 1, 114, 7)
1/1 [==============================] - 0s 69ms/step
1.0 7.116663e-18
(1, 1, 112, 7)
1/1 [==============

1/1 [==============================] - 0s 49ms/step
1.0 1.7469573e-17
(1, 1, 116, 7)
1/1 [==============================] - 0s 54ms/step
1.0 7.853151e-19
(1, 1, 116, 7)
1/1 [==============================] - 0s 49ms/step
1.0 5.70528e-19
(1, 1, 111, 7)
1/1 [==============================] - 0s 49ms/step
1.0 3.5730113e-19
(1, 1, 113, 7)
1/1 [==============================] - 0s 55ms/step
1.0 1.2079315e-19
(1, 1, 115, 7)
1/1 [==============================] - 0s 49ms/step
1.0 1.3394832e-19
(1, 1, 113, 7)
1/1 [==============================] - 0s 54ms/step
1.0 7.733899e-21
(1, 1, 111, 7)
1/1 [==============================] - 0s 56ms/step
1.0 4.1682778e-20
(1, 1, 113, 7)
1/1 [==============================] - 0s 66ms/step
1.0 7.759646e-19
(1, 1, 109, 7)
1/1 [==============================] - 0s 73ms/step
1.0 7.9944887e-19
(1, 1, 114, 7)
1/1 [==============================] - 0s 48ms/step
1.0 4.0724674e-19
(1, 1, 113, 7)
1/1 [==============================] - 0s 55ms/step
1.0 7.282731e-19
(

(1, 1, 112, 7)
1/1 [==============================] - 0s 54ms/step
1.0 1.5828518e-12
(1, 1, 114, 7)
1/1 [==============================] - 0s 54ms/step
1.0 9.757206e-14
(1, 1, 109, 7)
1/1 [==============================] - 0s 48ms/step
1.0 1.1274931e-12
(1, 1, 116, 7)
1/1 [==============================] - 0s 49ms/step
1.0 7.1793277e-13
(1, 1, 112, 7)
1/1 [==============================] - 0s 49ms/step
1.0 6.9368697e-13
(1, 1, 116, 7)
1/1 [==============================] - 0s 53ms/step
1.0 4.1535867e-13
(1, 1, 109, 7)
1/1 [==============================] - 0s 58ms/step
1.0 5.9818833e-13
(1, 1, 117, 7)
1/1 [==============================] - 0s 49ms/step
1.0 3.2390438e-14
(1, 1, 113, 7)
1/1 [==============================] - 0s 55ms/step
1.0 1.5901641e-14
(1, 1, 111, 7)
1/1 [==============================] - 0s 53ms/step
1.0 5.298533e-13
(1, 1, 112, 7)
1/1 [==============================] - 0s 48ms/step
1.0 1.13213855e-13
(1, 1, 112, 7)
1/1 [==============================] - 0s 54ms/step

1/1 [==============================] - 0s 55ms/step
1.0 1.0706449e-13
(1, 1, 112, 7)
1/1 [==============================] - 0s 54ms/step
1.0 2.7923437e-13
(1, 1, 112, 7)
1/1 [==============================] - 0s 51ms/step
1.0 1.317559e-13
(1, 1, 110, 7)
1/1 [==============================] - 0s 54ms/step
1.0 1.6849136e-13
(1, 1, 114, 7)
1/1 [==============================] - 0s 53ms/step
1.0 3.17803e-11
(1, 1, 122, 7)
1/1 [==============================] - 0s 48ms/step
1.0 1.10187744e-11
(1, 1, 114, 7)
1/1 [==============================] - 0s 73ms/step
1.0 1.2935929e-11
(1, 1, 114, 7)
1/1 [==============================] - 0s 49ms/step
1.0 2.548115e-12
(1, 1, 109, 7)
1/1 [==============================] - 0s 75ms/step
1.0 4.8082953e-12
(1, 1, 113, 7)
1/1 [==============================] - 0s 53ms/step
1.0 3.8173384e-12
(1, 1, 111, 7)
1/1 [==============================] - 0s 54ms/step
1.0 6.598484e-13
(1, 1, 114, 7)
1/1 [==============================] - 0s 53ms/step
1.0 1.089063e-11


1/1 [==============================] - 0s 65ms/step
1.0 1.8190177e-16
(1, 1, 115, 7)
1/1 [==============================] - 0s 46ms/step
1.0 6.8718703e-16
(1, 1, 114, 7)
1/1 [==============================] - 0s 56ms/step
1.0 5.5877715e-16
(1, 1, 116, 7)
1/1 [==============================] - 0s 59ms/step
1.0 7.888081e-16
(1, 1, 111, 7)
1/1 [==============================] - 0s 51ms/step
1.0 1.1522186e-15
(1, 1, 118, 7)
1/1 [==============================] - 0s 69ms/step
1.0 9.877391e-16
(1, 1, 112, 7)
1/1 [==============================] - 0s 74ms/step
1.0 1.703317e-15
(1, 1, 115, 7)
1/1 [==============================] - 0s 52ms/step
1.0 5.387932e-16
(1, 1, 109, 7)
1/1 [==============================] - 0s 55ms/step
1.0 2.1334085e-15
(1, 1, 116, 7)
1/1 [==============================] - 0s 68ms/step
1.0 3.2617685e-16
(1, 1, 112, 7)
1/1 [==============================] - 0s 68ms/step
1.0 1.5376217e-15
(1, 1, 117, 7)
1/1 [==============================] - 0s 49ms/step
1.0 1.4971689e-15

...

1/1 [==============================] - 0s 75ms/step
1.0 8.4277045e-20
(1, 1, 109, 7)
1/1 [==============================] - 0s 48ms/step
1.0 9.6072065e-20
(1, 1, 109, 7)
1/1 [==============================] - 0s 50ms/step
1.0 1.4050093e-19
(1, 1, 110, 7)
1/1 [==============================] - 0s 55ms/step
1.0 8.3602375e-20
(1, 1, 111, 7)
1/1 [==============================] - 0s 69ms/step
1.0 9.190755e-20
(1, 1, 111, 7)
1/1 [==============================] - 0s 68ms/step
1.0 9.8808753e-20
(1, 1, 109, 7)
1/1 [==============================] - 0s 75ms/step
1.0 4.9039803e-20
(1, 1, 109, 7)
1/1 [==============================] - 0s 51ms/step
1.0 2.0200437e-19
(1, 1, 119, 7)
1/1 [==============================] - 0s 54ms/step
1.0 5.563513e-20
(1, 1, 108, 7)
1/1 [==============================] - 0s 54ms/step
1.0 8.160282e-20
(1, 1, 110, 7)
1/1 [==============================] - 0s 52ms/step
1.0 5.772859e-20
(1, 1, 108, 7)
1/1 [==============================] - 0s 55ms/step
1.0 9.124243e-20


1/1 [==============================] - 0s 55ms/step
1.0 1.2259805e-19
(1, 1, 107, 7)
1/1 [==============================] - 0s 61ms/step
1.0 4.4008157e-20
(1, 1, 113, 7)
1/1 [==============================] - 0s 58ms/step
1.0 9.813523e-20
(1, 1, 102, 7)
1/1 [==============================] - 0s 59ms/step
1.0 1.1294218e-19
(1, 1, 112, 7)
1/1 [==============================] - 0s 58ms/step
1.0 1.6372422e-19
(1, 1, 115, 7)
1/1 [==============================] - 0s 52ms/step
1.0 1.1347986e-19
(1, 1, 110, 7)
1/1 [==============================] - 0s 61ms/step
1.0 1.05284136e-19
(1, 1, 111, 7)
1/1 [==============================] - 0s 53ms/step
1.0 1.2703276e-19
(1, 1, 111, 7)
1/1 [==============================] - 0s 55ms/step
1.0 1.1988697e-19
(1, 1, 114, 7)
1/1 [==============================] - 0s 61ms/step
1.0 5.388949e-20
(1, 1, 116, 7)
1/1 [==============================] - 0s 60ms/step
1.0 1.9852694e-19
(1, 1, 112, 7)
1/1 [==============================] - 0s 76ms/step
1.0 1.7595089e

1/1 [==============================] - 0s 64ms/step
1.0 4.3670415e-17
(1, 1, 111, 7)
1/1 [==============================] - 0s 72ms/step
1.0 2.447589e-15
(1, 1, 113, 7)
1/1 [==============================] - 0s 54ms/step
1.0 4.692863e-16
(1, 1, 110, 7)
1/1 [==============================] - 0s 49ms/step
1.0 4.7747724e-16
(1, 1, 108, 7)
1/1 [==============================] - 0s 48ms/step
1.0 3.856636e-17
(1, 1, 109, 7)
1/1 [==============================] - 0s 75ms/step
1.0 2.3284868e-15
(1, 1, 108, 7)
1/1 [==============================] - 0s 69ms/step
1.0 2.7421932e-16
(1, 1, 111, 7)
1/1 [==============================] - 0s 60ms/step
1.0 5.4996752e-15
(1, 1, 115, 7)
1/1 [==============================] - 0s 61ms/step
1.0 1.503709e-14
(1, 1, 115, 7)
1/1 [==============================] - 0s 57ms/step
1.0 1.4213246e-15
(1, 1, 106, 7)
1/1 [==============================] - 0s 56ms/step
1.0 1.9998674e-15
(1, 1, 119, 7)
1/1 [==============================] - 0s 63ms/step
1.0 4.198726e-13


1/1 [==============================] - 0s 50ms/step
1.0 6.835861e-18
(1, 1, 120, 7)
1/1 [==============================] - 0s 75ms/step
1.0 3.8803783e-18
(1, 1, 112, 7)
1/1 [==============================] - 0s 57ms/step
1.0 5.146493e-18
(1, 1, 110, 7)
1/1 [==============================] - 0s 48ms/step
1.0 3.8315147e-18
(1, 1, 117, 7)
1/1 [==============================] - 0s 69ms/step
1.0 3.781809e-18
(1, 1, 112, 7)
1/1 [==============================] - 0s 50ms/step
1.0 1.2204564e-18
(1, 1, 111, 7)
1/1 [==============================] - 0s 75ms/step
1.0 1.063681e-18
(1, 1, 112, 7)
1/1 [==============================] - 0s 56ms/step
1.0 5.4066833e-18
(1, 1, 104, 7)
1/1 [==============================] - 0s 58ms/step
1.0 4.7200402e-18
(1, 1, 116, 7)
1/1 [==============================] - 0s 62ms/step
1.0 5.232427e-18
(1, 1, 117, 7)
1/1 [==============================] - 0s 50ms/step
1.0 1.0543657e-17
(1, 1, 112, 7)
1/1 [==============================] - 0s 47ms/step
1.0 1.5682878e-17


(1, 1, 112, 7)
1/1 [==============================] - 0s 52ms/step
1.0 1.5545341e-18
(1, 1, 114, 7)
1/1 [==============================] - 0s 72ms/step
1.0 3.574281e-18
(1, 1, 112, 7)
1/1 [==============================] - 0s 67ms/step
1.0 2.757052e-19
(1, 1, 117, 7)
1/1 [==============================] - 0s 75ms/step
1.0 3.6057222e-19
(1, 1, 119, 7)
1/1 [==============================] - 0s 61ms/step
1.0 9.353888e-19
(1, 1, 108, 7)
1/1 [==============================] - 0s 74ms/step
1.0 1.8699624e-17
(1, 1, 114, 7)
1/1 [==============================] - 0s 55ms/step
1.0 3.147079e-19
(1, 1, 108, 7)
1/1 [==============================] - 0s 56ms/step
1.0 1.3950017e-18
(1, 1, 115, 7)
1/1 [==============================] - 0s 51ms/step
1.0 5.1739703e-19
(1, 1, 115, 7)
1/1 [==============================] - 0s 59ms/step
1.0 1.3101029e-18
(1, 1, 114, 7)
1/1 [==============================] - 0s 74ms/step
1.0 1.3625307e-18
(1, 1, 115, 7)
1/1 [==============================] - 0s 58ms/step
1.